<a href="https://colab.research.google.com/github/NILEG/TensorFlow/blob/main/NLP_009(Bigger_data_Text_Generation).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

In [2]:
!gdown --id 15UqmiIm0xwh9mt0IYq2z3jHaauxQSTQT

/usr/local/lib/python3.10/dist-packages/gdown/cli.py:121: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=15UqmiIm0xwh9mt0IYq2z3jHaauxQSTQT
To: /content/irish-lyrics-eof.txt
100% 69.0k/69.0k [00:00<00:00, 140MB/s]


In [ ]:
file=open("irish-lyrics-eof.txt")
data=file.read().lower().split("\n")
print(data[:10])

In [4]:
tokenizer=tf.keras.preprocessing.text.Tokenizer()
tokenizer.fit_on_texts(data)

In [ ]:
sentences=[]
for line in data:
  sequence=tokenizer.texts_to_sequences([line])[0]
  for i in range(1, len(sequence)):
    sentences.append(sequence[:i+1])
sentences[:10]

In [12]:
total_words=len(tokenizer.word_index)+1
max_len=max([len(x) for x in sentences])
max_len

16

In [ ]:
pad_sentences=tf.keras.preprocessing.sequence.pad_sequences(sentences, maxlen=max_len, padding="pre")
training_data=np.array(pad_sentences[:,:-1], dtype=np.float32)
labels=np.array(pad_sentences[:,-1], dtype=np.float32)
labels[:10]

In [15]:
model=tf.keras.Sequential([tf.keras.layers.Input(shape=(max_len-1, )),
                           tf.keras.layers.Embedding(total_words, 100),
                           tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(150)),
                           tf.keras.layers.Dense(total_words, activation="softmax")])
model.compile(optimizer=tf.keras.optimizers.Adam(lr=0.01), loss="sparse_categorical_crossentropy", metrics=["accuracy"])
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 15, 100)           269000    
                                                                 
 bidirectional_1 (Bidirecti  (None, 300)               301200    
 onal)                                                           
                                                                 
 dense_1 (Dense)             (None, 2690)              809690    
                                                                 
Total params: 1379890 (5.26 MB)
Trainable params: 1379890 (5.26 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [16]:
history=model.fit(training_data, labels, epochs=100)

Epoch 1/100
377/377 [==============================] - 22s 34ms/step - loss: 6.7621 - accuracy: 0.0640
Epoch 2/100
377/377 [==============================] - 5s 12ms/step - loss: 6.2306 - accuracy: 0.0751
Epoch 3/100
377/377 [==============================] - 4s 9ms/step - loss: 5.9514 - accuracy: 0.0835
Epoch 4/100
377/377 [==============================] - 3s 7ms/step - loss: 5.6463 - accuracy: 0.0994
Epoch 5/100
377/377 [==============================] - 3s 9ms/step - loss: 5.2994 - accuracy: 0.1167
Epoch 6/100
377/377 [==============================] - 3s 8ms/step - loss: 4.9313 - accuracy: 0.1373
Epoch 7/100
377/377 [==============================] - 3s 8ms/step - loss: 4.5560 - accuracy: 0.1589
Epoch 8/100
377/377 [==============================] - 3s 8ms/step - loss: 4.1961 - accuracy: 0.1867
Epoch 9/100
377/377 [==============================] - 3s 9ms/step - loss: 3.8545 - accuracy: 0.2228
Epoch 10/100
377/377 [==============================] - 3s 8ms/step - loss: 3.5243 - acc

####Highest Probability

In [31]:
seed_text="help me obi-wan kinobi youre my only hope"
seed_size=100
for i in range(seed_size):
  sequence=tokenizer.texts_to_sequences([seed_text])[0]
  pad_sequence=tf.keras.preprocessing.sequence.pad_sequences([sequence], maxlen=max_len-1, padding="pre")
  probabilities=model.predict(pad_sequence, verbose=0)
  index=np.argmax(probabilities, axis=-1)[0]
  if(index !=0):
    word=tokenizer.index_word[index]
    seed_text=seed_text+" "+word
print(seed_text)

help me obi-wan kinobi youre my only hope our more round our day day well more return nor laws leg smiling chop struggle souls to the fairy way to dublin true true can play the foemans did he stand he crew say you fell on the sod then i had had had a fine summers day had a jewel of skibbereen had a bishop time had had had had had sinking sinking island dove sinking guff bubblin bubblin stone fray rakes sea gesture rakes meadow pipes dawning paddys twas for the tree tree the broad down are down and down out by mooncoin town down down to glen


####Random Prediction from top 3 highest probabilities

In [33]:
seed_text="help me obi-wan kinobi youre my only hope"
seed_size=100
for i in range(seed_size):
  sequence=tokenizer.texts_to_sequences([seed_text])[0]
  pad_sequence=tf.keras.preprocessing.sequence.pad_sequences([sequence], maxlen=max_len-1, padding="pre")
  probabilities=model.predict(pad_sequence, verbose=0)

  choice=np.random.choice([1,2,3])

  index=np.argsort(probabilities)[0][-choice]
  if(index !=0):
    word=tokenizer.index_word[index]
    seed_text=seed_text+" "+word
print(seed_text)

help me obi-wan kinobi youre my only hope more turn for her more dear night return up in spirit her done for tralee away behind an fingers tree flower the rocky o rocky bells chop ill not could around the sod im be in fray skibbereen grow on your ground crossed gilgarra sea her fathers loo fondly sea thought day an day began to thought cant clergy dear fair sons sea vow morn be able nothing find primrose cant smoke thee sword on your banks and the bold theres a tear in your eye eyes ever ever down by mooncoin return by down again down to old her
